In [21]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [22]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of Lon

In [23]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [24]:
import nltk
from nltk.stem import PorterStemmer
stem = PorterStemmer()

In [25]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

31 documents
9 classes ['deliveryoption', 'goodbye', 'greeting', 'hours', 'location', 'menu', 'payments', 'thanks', 'todaysmenu']
59 unique lemmatized words ["'s", 'accept', 'address', 'anyone', 'are', 'bye', 'can', 'card', 'cash', 'course', 'credit', 'day', 'delicious', 'deliver', 'delivery', 'dish', 'do', 'food', 'for', 'from', 'good', 'goodbye', 'hello', 'helpful', 'hi', 'home', 'hour', 'how', 'is', 'later', 'located', 'location', 'main', 'mastercard', 'me', 'menu', 'most', 'only', 'open', 'option', 'provide', 'restaurant', 'see', 'serving', 'situated', 'special', 'take', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'today', 'what', 'when', 'where', 'you', 'your']


In [26]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stem.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\korla\Anaconda3\envs\TF\lib\site-packages\ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


### ANN Architectures Implementations

In [27]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
7/7 [==============================] - 0s 1ms/step - loss: 2.1860 - accuracy: 0.1290
Epoch 2/200
7/7 [==============================] - 0s 2ms/step - loss: 2.1627 - accuracy: 0.1290
Epoch 3/200
7/7 [==============================] - 0s 1ms/step - loss: 2.1766 - accuracy: 0.0323
Epoch 4/200
7/7 [==============================] - 0s 2ms/step - loss: 2.1168 - accuracy: 0.0968
Epoch 5/200
7/7 [==============================] - 0s 3ms/step - loss: 2.0415 - accuracy: 0.2258
Epoch 6/200
7/7 [==============================] - 0s 2ms/step - loss: 1.8996 - accuracy: 0.4516
Epoch 7/200
7/7 [==============================] - 0s 3ms/step - loss: 1.9799 - accuracy: 0.2258
Epoch 8/200
7/7 [==============================] - 0s 2ms/step - loss: 1.9601 - accuracy: 0.2903
Epoch 9/200
7/7 [==============================] - 0s 2ms/step - loss: 1.6873 - accuracy: 0.4516
Epoch 10/200
7/7 [==============================] - 0s 2ms/step - loss: 1.7446 - accuracy: 0.3548
Epoch 11/200
7/7 [===========

7/7 [==============================] - 0s 2ms/step - loss: 0.1554 - accuracy: 0.9032
Epoch 85/200
7/7 [==============================] - ETA: 0s - loss: 0.0360 - accuracy: 1.00 - 0s 2ms/step - loss: 0.0310 - accuracy: 1.0000
Epoch 86/200
7/7 [==============================] - 0s 3ms/step - loss: 0.2547 - accuracy: 0.9032
Epoch 87/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1824 - accuracy: 0.9355
Epoch 88/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1100 - accuracy: 0.9677
Epoch 89/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1425 - accuracy: 0.9677
Epoch 90/200
7/7 [==============================] - 0s 1ms/step - loss: 0.2261 - accuracy: 0.9677
Epoch 91/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1149 - accuracy: 0.9677
Epoch 92/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1527 - accuracy: 0.9355
Epoch 93/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1289 - accuracy

7/7 [==============================] - 0s 2ms/step - loss: 0.0677 - accuracy: 1.0000
Epoch 167/200
7/7 [==============================] - 0s 2ms/step - loss: 0.1546 - accuracy: 0.9677
Epoch 168/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1060 - accuracy: 0.9677
Epoch 169/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0675 - accuracy: 0.9677
Epoch 170/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0228 - accuracy: 1.0000
Epoch 171/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0285 - accuracy: 1.0000
Epoch 172/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0111 - accuracy: 1.0000
Epoch 173/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0400 - accuracy: 1.0000
Epoch 174/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0175 - accuracy: 1.0000
Epoch 175/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0836 - accuracy: 1.0000
Epoch 176/200
7/7 [=====

###  Predict the response

In [28]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

#### Logci for comparing the user imput with trained data 

In [29]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

### Logic for taking request and send response

In [30]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

### Testing our own data 

In [32]:
chatbot_response("hello there how are you")

'Hello, thanks for visiting'

In [33]:
chatbot_response("What is your Name")

'We are situated at the intersection of London Alley and Bridge Avenue'

In [34]:
chatbot_response("Hai")

'Have a nice day'